In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, ExtraTreesClassifier, AdaBoostClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from xgboost import XGBClassifier

In [57]:
ch=pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")
ch.drop(["customerID"],axis=1,inplace=True)

ch["TotalCharges"][ch["TotalCharges"]==" "]=0
ch["TotalCharges"]=ch["TotalCharges"].astype("float64")

#ch["tenure"][ch["Contract"]=="Month-to-month"]=0
#ch["tenure"][ch["Contract"]=="One year"]=1
#ch["tenure"][ch["Contract"]=="Two year"]=2

C:\Users\Barath\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [26]:
rf= RandomForestClassifier()
bc=BaggingClassifier()
etc=ExtraTreesClassifier()
abc=AdaBoostClassifier()
xgb=XGBClassifier()
lr=LogisticRegression(max_iter=1000)
knn=KNeighborsClassifier()
nb=GaussianNB()
tree=DecisionTreeClassifier()
#vc=VotingClassifier(estimators=[1000,2000,700,800,900,500,100])
gb= GradientBoostingClassifier()

model=[rf,xgb,lr,knn,nb,tree]
names=["Random forest","XG Boost","Logistic Regression","K Nearest Neighbors","Naive bayes","Decision Tree"]

In [7]:
ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]].apply(LabelEncoder().fit_transform)

In [59]:
x=ch[['tenure','PaperlessBilling_No','InternetService_Fiber optic','OnlineSecurity_No','TechSupport_No', 'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaymentMethod_Electronic check']]
y=ch["Churn"]

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2,random_state=478)

In [60]:
scores=pd.DataFrame(columns=["model","train score","accuracy score","recall score","precision score","tn","fp","tp","fn"])
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=i.predict(xtest)
    scores.loc[j]=[k,i.score(xtrain,ytrain),accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores

,model,train score,accuracy score,recall score,precision score,tn,fp,tp,fn
0,Random forest,0.889599,0.759404,0.532338,0.586301,856,151,214,188
1,XG Boost,0.808839,0.806246,0.539801,0.711475,919,88,217,185
2,Logistic Regression,0.799255,0.806955,0.542289,0.712418,919,88,218,184
3,K Nearest Neighbors,0.821796,0.770050,0.497512,0.621118,885,122,200,202
4,Naive bayes,0.720980,0.757275,0.855721,0.547771,723,284,344,58
5,Decision Tree,0.889599,0.754436,0.502488,0.580460,861,146,202,200


In [41]:
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=np.where(i.predict_proba(xtest)[:,1] > 0.25, 1, 0)
    scores.loc[j]=[k,i.score(xtrain,ytrain),accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores

,model,train score,accuracy score,recall score,precision score,tn,fp,tp,fn
0,Random forest,0.996983,0.753726,0.805970,0.546374,738,269,324,78
1,XG Boost,0.821086,0.763662,0.835821,0.557214,740,267,336,66
2,Logistic Regression,0.804224,0.772889,0.855721,0.567657,745,262,344,58
3,K Nearest Neighbors,0.832623,0.710433,0.639303,0.494231,744,263,257,145
4,Naive bayes,0.731984,0.762243,0.850746,0.554295,732,275,342,60
5,Decision Tree,0.996983,0.729595,0.514925,0.526718,821,186,207,195


## Parameter tuning & After parameter tuning

In [90]:
#KNN
from sklearn.model_selection import GridSearchCV
knn=KNeighborsClassifier()
param={"n_neighbors":np.arange(1,50),"weights":["uniform","distance"]}
CV_rfc= GridSearchCV(estimator=knn,param_grid=param, cv=3, scoring="f1_weighted")
CV_rfc.fit(xtrain,ytrain)
CV_rfc.best_params_

{'n_neighbors': 11, 'weights': 'uniform'}

In [91]:
tree=DecisionTreeClassifier()
param = {"criterion":["gini","entropy"]}
gs=GridSearchCV(tree,param,cv=4,scoring="f1_weighted")
gs.fit(xtrain,ytrain)
gs.best_params_

{'criterion': 'entropy'}

In [92]:
rfc= RandomForestClassifier(n_jobs=-1,max_features= "sqrt", n_estimators=50, oob_score= True)
param_grid={ "n_estimators":[200,700],"max_features":["auto","sqrt","log2"]}
CV_rfc= GridSearchCV(rfc,param_grid, cv=5)
CV_rfc.fit(xtrain,ytrain)
CV_rfc.best_params_

{'max_features': 'auto', 'n_estimators': 200}

In [96]:
rf= RandomForestClassifier(max_features="auto",n_estimators=200)
bc=BaggingClassifier(base_estimator=xgb, n_estimators=100)
etc=ExtraTreesClassifier()
abc=AdaBoostClassifier(base_estimator=lr,n_estimators=900,learning_rate=1)
xgb=XGBClassifier()
lr=LogisticRegression()
knn=KNeighborsClassifier(n_neighbors=11,weights="uniform")
nb=GaussianNB()
tree=DecisionTreeClassifier(criterion="entropy")
#vc=VotingClassifier(estimators=200)
gb= GradientBoostingClassifier(n_estimators=20, learning_rate=lr, max_features=2, max_depth=2)

model=[rf,bc,etc,abc,xgb,lr,knn,nb,tree,gb]
names=["Random forest","Bagging","Extra trees","adaboost","XG Boost","Logistic Regression","K Nearest Neighbors","Decision Tree","Gradient boosting"]

In [97]:
scores1=pd.DataFrame(columns=["index","model","accuracy score","recall score","precision score","tn","fp","tp","fn"])
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=i.predict(xtest)
    scores1.loc[j]=[j,k,accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores1

C:\Users\Barath\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Barath\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


,index,model,accuracy score,recall score,precision score,tn,fp,tp,fn
0,0,Random forest,0.801703,0.511664,0.652778,1648,175,329,314
1,1,Bagging,0.812652,0.539658,0.676413,1657,166,347,296
2,2,Extra trees,0.774939,0.486781,0.581784,1598,225,313,330
3,3,adaboost,0.739254,0.000000,0.000000,1823,0,0,643
4,4,XG Boost,0.811841,0.552100,0.668550,1647,176,355,288
5,5,Logistic Regression,0.804947,0.569207,0.642105,1619,204,366,277
6,6,K Nearest Neighbors,0.799270,0.452566,0.670507,1680,143,291,352
7,7,Decision Tree,0.737226,0.783826,0.497532,1314,509,504,139
8,8,Gradient boosting,0.733982,0.524106,0.490539,1473,350,337,306


In [103]:
rf= RandomForestClassifier(max_features="auto",n_estimators=200)
bc=BaggingClassifier(base_estimator=xgb, n_estimators=100)
etc=ExtraTreesClassifier()
abc=AdaBoostClassifier(base_estimator=lr,n_estimators=900,learning_rate=1)
xgb=XGBClassifier()
lr=LogisticRegression()
knn=KNeighborsClassifier(n_neighbors=11,weights="uniform")
nb=GaussianNB()
tree=DecisionTreeClassifier(criterion="entropy")
#vc=VotingClassifier(estimators=200)
gb= GradientBoostingClassifier(n_estimators=20, learning_rate=lr, max_features=2, max_depth=2)

model=[rf,bc,etc,abc,xgb,lr,knn,nb,tree,gb]
names=["Random forest","Bagging","Extra trees","adaboost","XG Boost","Logistic Regression","K Nearest Neighbors","Decision Tree","Gradient boosting"]

In [104]:
scores1=pd.DataFrame(columns=["index","model","accuracy score","recall score","precision score","tn","fp","tp","fn"])
j=0
for i,k in zip(model,names):
    i.fit(xtrain,ytrain)
    pred=i.predict(xtest)
    scores1.loc[j]=[j,k,accuracy_score(ytest,pred),recall_score(ytest,pred),precision_score(ytest,pred),confusion_matrix(ytest,pred)[0,0],confusion_matrix(ytest,pred)[0,1],confusion_matrix(ytest,pred)[1,1],confusion_matrix(ytest,pred)[1,0]]
    j=j+1
scores1

C:\Users\Barath\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\Barath\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic

,index,model,accuracy score,recall score,precision score,tn,fp,tp,fn
0,0,Random forest,0.802920,0.514774,0.655446,1649,174,331,312
1,1,Bagging,0.812247,0.545879,0.672414,1652,171,351,292
2,2,Extra trees,0.789538,0.493002,0.621569,1630,193,317,326
3,3,adaboost,0.813463,0.559876,0.670391,1646,177,360,283
4,4,XG Boost,0.811436,0.561431,0.663603,1640,183,361,282
5,5,Logistic Regression,0.812652,0.580093,0.660177,1631,192,373,270
6,6,K Nearest Neighbors,0.796026,0.452566,0.658371,1672,151,291,352
7,7,Decision Tree,0.759530,0.744946,0.527533,1394,429,479,164
8,8,Gradient boosting,0.742903,0.542768,0.506531,1483,340,349,294


In [58]:
ch[["Churn"]]=ch[["Churn"]].apply(LabelEncoder().fit_transform)
#ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]]=ch[["gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',"Churn"]].apply(LabelEncoder().fit_transform)

u=["gender","Partner","Dependents","PhoneService",'PaperlessBilling',"MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV",'StreamingMovies', 'Contract', 'PaymentMethod']
for i in u:
    m=ch[i].unique()
    m=m+i
    dummy = pd.get_dummies(ch[i],columns=ch[i].unique(),prefix=i)
    ch=pd.concat([ch,dummy],axis=1,sort=True)
    ch.drop([i],axis=1,inplace=True)
#ch.drop(['OnlineSecurity_No internet service',"OnlineBackup_No internet service","DeviceProtection_No internet service","TechSupport_No internet service","StreamingTV_No internet service","StreamingMovies_No internet service"],axis=1,inplace=True)

In [37]:
ch

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,1,29.85,29.85,0,1,0,0,1,1,...,0,1,0,1,0,0,0,0,1,0
1,0,34,56.95,1889.50,0,0,1,1,0,1,...,0,1,0,0,1,0,0,0,0,1
2,0,2,53.85,108.15,1,0,1,1,0,1,...,0,1,0,1,0,0,0,0,0,1
3,0,45,42.30,1840.75,0,0,1,1,0,1,...,0,1,0,0,1,0,1,0,0,0
4,0,2,70.70,151.65,1,1,0,1,0,1,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,24,84.80,1990.50,0,0,1,0,1,0,...,1,0,1,0,1,0,0,0,0,1
7039,0,72,103.20,7362.90,0,1,0,0,1,0,...,1,0,1,0,1,0,0,1,0,0
7040,0,11,29.60,346.45,0,1,0,0,1,0,...,0,1,0,1,0,0,0,0,1,0
7041,1,4,74.40,306.60,1,0,1,0,1,1,...,0,1,0,1,0,0,0,0,0,1


In [42]:
from sklearn.feature_selection import RFE
rfe=RFE(lr,10)
xrfe=rfe.fit_transform(x,y)
lr.fit(xrfe,y)
pd.DataFrame({"support":rfe.support_,"ranking":rfe.ranking_,"columns":list(x.columns)})

,support,ranking,columns
0,False,15,SeniorCitizen
1,False,21,tenure
2,False,27,MonthlyCharges
3,False,28,TotalCharges
4,False,23,Partner_No
5,False,24,Partner_Yes
6,False,12,Dependents_No
7,False,2,Dependents_Yes
8,False,13,PhoneService_No
9,False,9,PhoneService_Yes


In [44]:
from sklearn.feature_selection import RFE
rfe=RFE(xgb,10)
xrfe=rfe.fit_transform(x,y)
lr.fit(xrfe,y)
pd.DataFrame({"support":rfe.support_,"ranking":rfe.ranking_,"columns":list(x.columns)})

,support,ranking,columns
0,False,8,SeniorCitizen
1,True,1,tenure
2,False,5,MonthlyCharges
3,False,7,TotalCharges
4,False,16,Partner_No
5,False,25,Partner_Yes
6,False,9,Dependents_No
7,False,24,Dependents_Yes
8,False,11,PhoneService_No
9,False,28,PhoneService_Yes


In [55]:
from sklearn.feature_selection import RFE
rfe=RFE(nb,40)
xrfe=rfe.fit_transform(x,y)
lr.fit(xrfe,y)
pd.DataFrame({"support":rfe.support_,"ranking":rfe.ranking_,"columns":list(x.columns)})

,support,ranking,columns
0,True,1,SeniorCitizen
1,True,1,tenure
2,True,1,MonthlyCharges
3,True,1,TotalCharges
4,True,1,Partner_No
5,True,1,Partner_Yes
6,True,1,Dependents_No
7,True,1,Dependents_Yes
8,True,1,PhoneService_No
9,True,1,PhoneService_Yes
